In [3]:
import geemap
import ee
import os
import calendar
Map = geemap.Map()
Map

Enter verification code:  4/1AX4XfWhXipMIjL3jW8Px1dJT_CT0OkZoHX-Isl7NsTm8vkRjUKj3DxUfCO4



Successfully saved authorization token.


Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [2]:
lst_collections = ee.ImageCollection("MODIS/061/MOD11A1")
Map.addLayer(lst_collections, {}, 'lst_collections')
word_shp = "D:/图层/word_shp/word_polygon.shp"
word = geemap.shp_to_ee(word_shp)
roi = word.geometry()
print(word.getInfo())
Map.addLayer(word, {}, 'word')

{'type': 'FeatureCollection', 'columns': {'Id': 'Integer', 'system:index': 'String'}, 'features': [{'type': 'Feature', 'geometry': {'geodesic': False, 'type': 'Polygon', 'coordinates': [[[-180, 90], [-180, -90], [180, -90.00000000000003], [180, 89.99999999999997], [-180, 90]]]}, 'id': '0', 'properties': {'Id': 0}}]}


In [5]:
import datetime
def get_date_list(start_date, end_date, format,step):
    """
    根据开始日期、结束日期返回这段时间里所有天的集合
    :param start_date: 开始日期(日期格式或者字符串格式)
    :param end_date: 结束日期(日期格式或者字符串格式)
    :param format: 格式化字符串, 如: '%Y-%m-%d'
    :param step: int, 如: (1,8,30)
    :return:
    """
    date_list = []
    if isinstance(start_date, str) and isinstance(end_date, str):
        start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
        end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')
    date_list.append(start_date.strftime(format))
    while start_date < end_date:
        start_date += datetime.timedelta(days=step)
        date_list.append(start_date.strftime(format))
    return date_list

In [3]:
def mul_scal(image):
    return image.multiply(0.02)

In [7]:
for year in range(2000,2019):
    if year ==2000:
        month = '02'
        day = '24'
    else:
        month = '01'
        day = '01'
    start_year = str(year) + '-' + month + '-' + day
    end_year = str(year+1) + '-' + '01' + '-' + '01'
    year_list = get_date_list(start_year,end_year,'%Y-%m-%d',8)
    for name ,data in enumerate(year_list[:-1]):
        start_data = year_list[name]
        end_data = year_list[name+ 1]
        lst_image_8day = lst_collections.filterDate(start_data,end_data).select('LST_Day_1km')
        #print('start_data:  ',start_data)
        print(f'{start_data}-{end_data}:  ',lst_image_8day.size().getInfo())
        if lst_image_8day.size().getInfo()==0:
            print('lst_image_8day is 0, continue---------')
            continue
        lst_8day_mean = lst_image_8day.map(mul_scal).mean().clip(roi)
        lst_resolution = lst_8day_mean.projection().nominalScale()
        #print('lst_resolution.getInfo()',lst_resolution.getInfo())
        filename = r'E:\EQI_data_prepare\LST\MODIS' + os.sep + str(year) + os.sep + start_data + '.tif'
        print('filename:  ',filename)
        geemap.ee_export_image(lst_8day_mean, filename = filename, scale=int(round(lst_resolution.getInfo())),region=roi,file_per_band=False)
        # while not os.path.exists(filename):
        #     clip_dow_merge(roi, lst_8day_mean, filename, int(round(lst_resolution.getInfo())))
        print(f'{filename} is ok ')
    print(f'{year} is ok ')
print('---------all is ok---------------')

2000-02-24-2000-03-03:   8
filename:   E:\EQI_data_prepare\LST\MODIS\2000\2000-02-24.tif
Generating URL ...
Please wait ...
Data downloaded to E:\EQI_data_prepare\LST\MODIS\2000\2000-02-24.tif
E:\EQI_data_prepare\LST\MODIS\2000\2000-02-24.tif is ok 
2000-03-03-2000-03-11:   8
filename:   E:\EQI_data_prepare\LST\MODIS\2000\2000-03-03.tif
Generating URL ...
Please wait ...
Data downloaded to E:\EQI_data_prepare\LST\MODIS\2000\2000-03-03.tif
E:\EQI_data_prepare\LST\MODIS\2000\2000-03-03.tif is ok 
2000-03-11-2000-03-19:   8
filename:   E:\EQI_data_prepare\LST\MODIS\2000\2000-03-11.tif
Generating URL ...
Please wait ...
Data downloaded to E:\EQI_data_prepare\LST\MODIS\2000\2000-03-11.tif
E:\EQI_data_prepare\LST\MODIS\2000\2000-03-11.tif is ok 
2000-03-19-2000-03-27:   8
filename:   E:\EQI_data_prepare\LST\MODIS\2000\2000-03-19.tif
Generating URL ...
Please wait ...
Data downloaded to E:\EQI_data_prepare\LST\MODIS\2000\2000-03-19.tif
E:\EQI_data_prepare\LST\MODIS\2000\2000-03-19.tif is ok 


<class 'UnboundLocalError'>: local variable 'r' referenced before assignment